In [112]:
import pandas as pd

In [113]:
#Чтение файлов:
df1T = pd.read_excel('Источники/transactions_data.xlsx')
df2C = pd.read_json('Источники/clients_data.json')  

In [114]:
#Проверки на дубли:
df1T.duplicated().sum()
df2C.duplicated().sum()

np.int64(0)

In [130]:
#Проверка на пустые значения:
df1T.isna().sum() #Проверка на Пустые значения в Транзакциях. 

#UPD1: Значения по 0 должно быть, после очистки. 
#UPD2: Есть данные где остальные присутствуют, но нет дат. Заполенено "INVALID_DATE".

transaction_id      0
client_id           0
transaction_date    0
service             0
amount              0
payment_method      0
city                0
consultant          0
dtype: int64

In [116]:
#Проверка на пустые значения:
df2C.isna().sum() #Проверка на Пустые значения в Клиентах.

id              1
age           983
gender       3319
net_worth     475
dtype: int64

In [117]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Провоерка выше.

In [118]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [119]:
#Замена INVALID_DATE на Nan

df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'], errors='coerce')

#Дает ошибку, если за ранее не изменнить.
#"Unknown datetime string format, unable to parse: INVALID_DATE, at position 15".

In [120]:
#Преобразование дат к одному в формату с целью проверки изменения значения:
df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'])
#Приведение всех дат к одному формату с точностью до секунд:
df1T['transaction_date'] = df1T['transaction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [121]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

Есть файлы где нет даты, но остальные в норме. Продолжим удаление, но по условиям. 
*Открыл вручуную файл Excel.*. 

In [122]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Пересмотрел план, удаляю данные где нет даты. Так как возможно будет мешать прогнозированию.

In [123]:
#Удаление строк с "Неизвестной услугой":
df1T = df1T[df1T['service'] != 'Неизвестная услуга']

In [124]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [125]:
#Продолжаем очистку, теперь зайдем со стороны аномальных значений:
#Использую "Межквартильный размах" (IQR). 

# Рассчитываем первый и третий квартели, чтобы посчитать IQR.
Q1 = df1T['amount'].quantile(0.25)
Q3 = df1T['amount'].quantile(0.75)

#Расчитываем IQR:
IQR = Q3 - Q1

#Устанавливаем границы 
lower_bound = max(0, Q1 - 1.5 * IQR) #Нижняя. Нижняя не может быть меньше 0. (...отрицательные суммы).
upper_bound = Q3 + 1.5 * IQR #Верхняя.

#Вывод границ: 
print(lower_bound)
print(upper_bound)

0
149823.31445055085


In [126]:
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]
print(anomal[['service','amount']])

                              service         amount
65                Управление активами  165531.626439
191   Инвестиционное консультирование  176346.818037
251   Инвестиционное консультирование  158548.065675
272               Управление активами  163445.690198
347           Финансовое планирование  194496.625320
...                               ...            ...
9345           Налоговое планирование  150878.174850
9500        Структурирование капитала  158543.668647
9631        Структурирование капитала  168401.012948
9643           Налоговое планирование  160444.593254
9671  Инвестиционное консультирование  161210.845033

[134 rows x 2 columns]


In [127]:
#Удаление аномальных значений: 
df1T = df1T[(df1T['amount'] >= lower_bound) & (df1T['amount'] <= upper_bound)] 

In [128]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [129]:
#Проверка: 
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]

print(anomal)

Empty DataFrame
Columns: [transaction_id, client_id, transaction_date, service, amount, payment_method, city, consultant]
Index: []
